This script tests the joint weight and noise variance optimization procedure of Private ColRel using gradient descent based optimization algorithms

In [ ]:
# Requirements

import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
np.set_printoptions(precision=2)

torch.manual_seed(0)

In [ ]:
# mmWave network topology definition

def client_locations_mmWave_clusters_intermittent(num_clients: int = 10):
    """
    :param num_clients: Number of federated edge learning clients
    :return: probability of successful transmission to PS, and inter-client connectivity matrix
    """

    # PS is placed at origin
    PS_loc = np.array([0, 0])

    # Distance of good clients from PS
    circle_good_rad = 159                       # meters. For prob success ~ 0.9

    # Clients angles
    client_vec_deg = np.zeros(num_clients)
    client_vec_deg[3] = 2 * np.pi / 3
    client_vec_deg[6] = 4 * np.pi / 3

    x = np.zeros(num_clients)
    y = np.zeros(num_clients)

    # Determining the Cartesian coordinates of the clients with good connectivity
    x[0] = circle_good_rad * np.cos(client_vec_deg[0])
    y[0] = circle_good_rad * np.sin(client_vec_deg[0])
    x[3] = circle_good_rad * np.cos(client_vec_deg[3])
    y[3] = circle_good_rad * np.sin(client_vec_deg[3])
    x[6] = circle_good_rad * np.cos(client_vec_deg[6])
    y[6] = circle_good_rad * np.sin(client_vec_deg[6])

    d = 159                       # Distance of bad clients in each cluster to the good client of the cluster

    # Cluster 1
    ang1 = 1.582
    x[1] = x[0] + d * np.cos(ang1)
    y[1] = y[0] + d * np.sin(ang1)
    print("Distance = {}".format(np.sqrt(x[1] ** 2 + y[1] ** 2)))

    ang2 = - 1.582
    x[2] = x[0] + d * np.cos(ang2)
    y[2] = y[0] + d * np.sin(ang2)

    # Cluster 2
    ang4 = 2 * np.pi / 3 + 1.582
    x[4] = x[3] + d * np.cos(ang4)
    y[4] = y[3] + d * np.sin(ang4)

    ang5 = 2 * np.pi / 3 - 1.582
    x[5] = x[3] + d * np.cos(ang5)
    y[5] = y[3] + d * np.sin(ang5)

    # Cluster 3
    ang7 = 4 * np.pi / 3 - 1.582
    x[7] = x[6] + d * np.cos(ang7)
    y[7] = y[6] + d * np.sin(ang7)

    ang8 = 4 * np.pi / 3 - 1.54
    x[8] = x[6] + d * np.cos(ang8)
    y[8] = y[6] + d * np.sin(ang8)

    ang9 = 4 * np.pi / 3 + 1.582
    x[9] = x[6] + d * np.cos(ang9)
    y[9] = y[6] + d * np.sin(ang9)

    # Client locations
    loc_clients = []
    for idx in range(num_clients):
        loc_clients.append([x[idx], y[idx]])
    loc_clients = np.array(loc_clients)

    # Compute distances of clients from PS
    sub_clients_PS = loc_clients - PS_loc
    dist_clients_PS2 = np.zeros(len(sub_clients_PS))
    for idx in range(len(sub_clients_PS)):
        dist_clients_PS2[idx] = sub_clients_PS[idx][0] ** 2 + sub_clients_PS[idx][1] ** 2
    dist_clients_PS = np.sqrt(dist_clients_PS2)

    # Compute pairwise distances between clients
    dist_clients_clients = np.zeros([num_clients, num_clients])
    for i in range(len(loc_clients)):
        for j in range(len(loc_clients)):
            dist_vector = loc_clients[i] - loc_clients[j]
            dist_clients_clients[i][j] = np.linalg.norm(dist_vector, 2)

    # Compute probability of successful transmission to PS
    prob_success_PS = np.zeros(num_clients)
    for i in range(len(dist_clients_PS)):
        p = min(1, np.exp(-dist_clients_PS[i] / 30 + 5.2))
        prob_success_PS[i] = np.round(p * 100) / 100

    # Determine connectivity amongst clients
    P = np.zeros([num_clients, num_clients])
    connectivity_mat_clients = np.zeros([num_clients, num_clients])
    for i in range(num_clients):
        for j in range(num_clients):
            p = min(1, np.exp(-dist_clients_clients[i][j] / 30 + 5.2))
            if p > 0.5:
                P[i][j] = p
                connectivity_mat_clients[i][j] = 1

    with open('prob_success_PS.npy', 'wb') as f:
        np.save(f, prob_success_PS)
    with open('connectivity_mat_clients.npy', 'wb') as f:
        np.save(f, connectivity_mat_clients)

    return prob_success_PS, P, connectivity_mat_clients

In [ ]:
# Topology induced variance

def evaluate_tiv(p: torch.Tensor = None, A: torch.Tensor = None, P: torch.Tensor = None, radius: float = 1.0):
    """Evaluate the upper bound on topology induced variance
        :param p: Array of transmission probabilities from each of the clients to the PS
        :param A: Matrix of weights
        :param P: Matrix of probabilities for intermittent connectivity amongst clients
        :param radius: Radius of the Euclidean ball in which the data vectors lie
    """
    
    num_clients = len(p)
    
    # Validate inputs
    assert num_clients == A.shape[0] == A.shape[1], "p and A dimension mismatch!"
    assert num_clients == P.shape[0] == P.shape[1], "p and P dimension mismatch!"
    
    # First term
    S = 0

    # First term
    for i in range(num_clients):
        for l in range(num_clients):
            for j in range(num_clients):
                S += p[j] * (1 - p[j]) * P[i][j] * P[l][j] * A[i][j] * A[l][j]

    # Second term
    for i in range(num_clients):
        for j in range(num_clients):
            S += P[i][j] * p[j] * (1 - P[i][j]) * A[i][j] * A[i][j]

    # Third term
    for i in range(num_clients):
        for l in range(num_clients):
            assert P[i][l] == P[l][i], "Matrix P must be symmetric."
            E = P[i][l]
            S += p[i] * p[l] * (E - P[i][l] * P[l][i]) * A[l][i] * A[i][l]
            
    # Compute the bias terms
    s = torch.zeros(num_clients)
    for i in range(num_clients):
        for j in range(num_clients):
            s[i] += p[j] * P[i][j] * A[i][j]
            
    # Contribution of the term due to bias
    for i in range(num_clients):
        for j in range(num_clients):
            S += (s[i]*s[j] - 2*s[i] + 1)

    return S * (radius ** 2) / num_clients ** 2


In [ ]:
# Privacy induced variance

def evaluate_piv(p: torch.Tensor = None, P: torch.Tensor = None, sigma: torch.Tensor = None, dimension: int = 128):
    """Evaluate the upper bound on privacy induced variance
        :param p: Array of transmission probabilities from each of the clients to the PS
        :param P: Matrix of probabilities for intermittent connectivity amongst clients
        :param sigma: Matrix of privacy noise variance between pairs of nodes
        :param dimension: Dimension of the data vectors
    """
    
    num_clients = len(p)
    
    # Validate inputs
    assert num_clients == P.shape[0] == P.shape[1], "p and P dimension mismatch!"
    assert num_clients == sigma.shape[0] == sigma.shape[1], "p and sigma dimension mismatch!"

    piv = 0

    for i in range(num_clients):
        for j in range(num_clients):
            piv += p[j] * P[i][j] * sigma[i][j] ** 2
            
    return piv * dimension / (num_clients ** 2)

In [ ]:
# Evaluate mean square error

def evaluate_mse(p: torch.Tensor = None, A: torch.Tensor = None, P: torch.Tensor = None, 
                 radius: float = 1.0, sigma: torch.Tensor = None, dimension: int = 128):
    """Evaluate the MSE
        :param p: Array of transmission probabilities from each of the clients to the PS
        :param A: Matrix of weights
        :param P: Matrix of probabilities for intermittent connectivity amongst clients
        :param radius: Radius of the Euclidean ball in which the data vectors lie
        :param sigma: Matrix of privacy noise variance between pairs of nodes
        :param dimension: Dimension of the data vectors
    """

    return evaluate_tiv(p=p, A=A, P=P, radius=radius) + evaluate_piv(p=p, P=P, sigma=sigma, dimension=dimension)
    

In [ ]:
# Regularization to control bias

def bias_regularizer(p: torch.Tensor = None, A: torch.Tensor = None, P: torch.Tensor = None, 
                         reg_type: str = "L2", reg_strength: torch.Tensor = torch.tensor(0)):
    """Evaluate the regularization term
        :param p: Array of transmission probabilities from each of the clients to the PS.
        :param A: Matrix of weights
        :param P: Matrix of probabilities for intermittent connectivity amongst clients
        :param reg_type: Type of regularization
        :param reg_strength: Hyperparameter -- weight of regularization term
    """
    
    num_clients = len(p)
    A_dim = A.shape
    neighbors_dim = P.shape
    
    # Validate inputs
    assert num_clients == A_dim[0] == A_dim[1]
    assert num_clients == neighbors_dim[0] == neighbors_dim[1]
    
    if reg_type == "L2":
        
        # Compute the bias terms
        bias = torch.zeros(num_clients)
        for i in range(num_clients):
            for j in range(num_clients):
                bias[i] += p[j] * P[i][j] * A[i][j]
            bias[i] -= 1
            
        return reg_strength / 2.0 * torch.norm(bias, 2) ** 2 
    
    elif reg_type == "L1":
        
        # Compute the bias terms
        bias = torch.zeros(num_clients)
        for i in range(num_clients):
            for j in range(num_clients):
                bias[i] += p[j] * P[i][j] * A[i][j]
            bias[i] -= 1
            
        return reg_strength / 2.0 * torch.norm(bias, 1) ** 2
    

In [ ]:
# Torch optimization

class Model(nn.Module):
    """Custom Pytorch model for gradient optimization.
    """
    def __init__(self, p: torch.Tensor = None, P: torch.Tensor = None, E: torch.Tensor = None, 
                 D: torch.Tensor = None, radius: float = 1.0,  dimension: int = 128,
                 reg_type: str = "L2", reg_strength: float = 0):
        
        super().__init__()
        num_clients = len(p)

        # Validate inputs
        assert num_clients == P.shape[0] == P.shape[1], "p and P dimension mismatch!"
        assert num_clients == E.shape[0] == E.shape[1], "p and E dimension mismatch!"
        assert num_clients == D.shape[0] == D.shape[1], "p and D dimension mismatch!"
        
        # Initialize weights with random numbers consistent with network topology
        weights = torch.distributions.Uniform(0, 0.1).sample((num_clients, num_clients))
        weights = torch.where(P > 0, weights, 0)

        # Initialize sigma consistent with the privacy constraint
        noise = torch.zeros_like(weights)
        for i in range(num_clients):
            for j in range(num_clients):
                noise[i][j] = 2 * weights[i][j] * radius / E[i][j] * torch.sqrt(2 * torch.log(1.25 / D[i][j]))
        
        # Make weights and privacy variances torch parameters
        self.weights = nn.Parameter(weights) 
        self.noise = nn.Parameter(noise)
        self.p = p
        self.P = P
        self.E = E
        self.D = D
        self.radius = radius
        self.dimension = dimension
        self.reg_type = reg_type
        self.reg_strength = reg_strength

        # Compute slopes / privacy coefficients
        B = torch.zeros_like(E)
        for i in range(num_clients):
            for j in range(num_clients):
                B[i][j] = 2 * radius / E[i][j] * torch.sqrt(2 * torch.log(1.25 / D[i][j]))

        self.B = B      
        
    def forward(self):
        """Implement the topology induced variance to be optimized
        """
        
        p = self.p
        P = self.P
        A = self.weights
        sigma = self.noise
        radius = self.radius
        dimension = self.dimension
        reg_type = self.reg_type
        reg_strength = self.reg_strength

        forward_loss = evaluate_mse(p=p, A=A, P=P, radius=radius, sigma=sigma, dimension=dimension) 
        + bias_regularizer(p=p, A=A, P=P, reg_type=reg_type, reg_strength=reg_strength)

        return forward_loss

In [ ]:
# Projecting to a cone to respect privacy and non-negativity constraints

class ConeProjector(object):
    """Project the weights and the noise to a cone
    """

    def __init__(self, frequency=1):
        self.frequency = frequency

    def __call__(self, module):
            
        A = module.weights.data
        sigma = module.noise.data
        num_clients = A.shape[0]
        B = module.B

        for i in range(num_clients):
            for j in range(num_clients):

                if sigma[i][j] >= 0 and A[i][j] < 0:
                    A[i][j] = 0

                elif sigma[i][j] < 0 or (sigma[i][j] >= 0 and sigma[i][j] < B[i][j] * A[i][j]):
                    A[i][j] = max(A[i][j] + B[i][j] * sigma[i][j], 0) / (1 + B[i][j] ** 2)
                    sigma[i][j] = B[i][j] * A[i][j]

In [ ]:
# Optimization loop

cone_projector = ConeProjector()

def training_loop(model: nn.Module, optimizer, n: int = 1500):
    "Training loop for torch model."
    losses = []
    
    for i in range(n):
        
        loss = model()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if i % cone_projector.frequency == 0:
            model.apply(cone_projector)
            
        losses.append(loss) 
        
        if i % 100 == 0:
            print(f"Iteration: {i}/{n}")
        
    return losses

In [ ]:
# Instantiate model and network topology
num_clients = 5
radius = 1.0
dimension = 128

# Connectivity parameters
p = torch.Tensor([0.5, 0.4, 0.3, 0.8, 0.9])
P = 0.9 * torch.ones(num_clients, num_clients)
P.fill_diagonal_(1)

# # Privacy parameters (No privacy)
# delta = 1e-3
# D = delta * torch.ones([num_clients, num_clients])
# eps = 1e3
# E = eps * torch.ones([num_clients, num_clients])

# Privacy parameters (Nodes trust their immediate one-hop neighbor)
delta = 1e-3
D = delta * torch.ones([num_clients, num_clients])
eps1 = 1e3             # For oneself and trustworthy neighbors
eps2 = 1            # For general neighbors
E = eps2 * torch.ones([num_clients, num_clients])
for i in range(num_clients):
    E[i][i] = eps1
    E[i][(i + 1) % num_clients] = eps1
    E[i][(i - 1) % num_clients] = eps1

# Bias control paramters
reg_type = "L1"
reg_strength = 0.0

# num_clients = 10
# p, P, _ = client_locations_mmWave_clusters_intermittent(num_clients)
# print(f"p = {p}")
# print(f"P = \n{P}")
# p = torch.tensor(p)
# P = torch.tensor(P)
# reg_type = "L1"
# reg_strength = torch.tensor(0)

In [ ]:
# Do the optimization

m = Model(p=p, P=P, E=E, D=D, radius=radius, dimension=dimension, reg_type=reg_type, reg_strength=reg_strength)
opt = torch.optim.Adam(m.parameters(), lr=0.001)
losses = training_loop(m, opt)
plt.figure(figsize=(14, 7))
with torch.no_grad():
    plt.plot(losses)

print("Collaboration weights are:\n")
print(m.weights)
print("Privacy noises are:\n")
print(m.noise)


In [ ]:
print(m.B)

In [ ]:
print(f"Final loss: {losses[-1]}")

In [ ]:
W = m.weights.detach().numpy()

In [ ]:
print(evaluate_tiv(p, W, P))

In [ ]:
print("Check biasedness for each node: \n")

bias = torch.zeros(num_clients)
for i in range(num_clients):
    for j in range(num_clients):
        bias[i] += p[j] * P[i][j] * W[i][j]
    bias[i] -= 1
    print(f"Bias at node {i} is {bias[i]}")

In [ ]:
# Clipping to ensure non-negative weights

class ZeroClipper(object):
    """Clip the weights to zero if they are negative
    """

    def __init__(self, frequency=1):
        self.frequency = frequency

    def __call__(self, module):
        # filter the variables to get the ones you want
        if hasattr(module, 'weights'):
            w = module.weights.data
            nn.ReLU(inplace=True)(w)